Inspired by https://bergvca.github.io/2017/10/14/super-fast-string-matching.html

In [54]:
import sys
import os

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path+"/scripts")

In [92]:
import pandas as pd
import pickle

with open("../data/all_reviews.pkl","rb") as f:
    reviews = pickle.load(f)
    

In [101]:
# with open("../data/job_titles.txt","r") as f:
#     common_tech_titles = [ l.replace("\n","").strip() for l in f ]
import string
import re
from DataPrep import cleanTitle


common_tech_titles = pd.read_csv("../data/job_titles.txt")

common_tech_titles = common_tech_titles.Job_Title.apply(cleanTitle).tolist()
common_tech_titles = list(set(common_tech_titles))


display(common_tech_titles)


['cloud system engineer',
 'technology analyst',
 'technology coordinator',
 'network systems administrator',
 'telecommunications specialist',
 'system analyst',
 'chief information officer',
 'data quality manager',
 'computer systems manager',
 'system designer',
 'chief technology officer',
 'technical support engineer',
 'technical specialist',
 'cloud consultant',
 'cloud architect',
 'customer support specialist',
 'applications engineer',
 'cloud system administrator',
 'associate developer',
 'front end developer',
 'web administrator',
 'java developer',
 'system administrator',
 'software engineer',
 'information security',
 'computer network architect',
 'cloud product project manager',
 'network computer systems administrator',
 'cloud services developer',
 'technology support specialist',
 'systems analyst',
 'software quality assurance analyst',
 'software architect',
 'systems administrator',
 'computer information research scientist',
 'desktop support manager',
 'netw

## N-Gram with Top N Cosine

In [57]:
# clean_job_titles = reviews.clean_job_title.tolist()
clean_job_titles_filtered = reviews.clean_job_title.value_counts()
clean_job_titles_filtered = clean_job_titles_filtered.loc[clean_job_titles_filtered > 1].index.tolist()

clean_job_titles_filtered.extend(common_tech_titles)

print(len(clean_job_titles_filtered))

106249


In [58]:
import re

def ngrams(string, n=3):
    #string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

print('All 3-grams in {}:'.format(clean_job_titles_filtered[0]))
print( ngrams(clean_job_titles_filtered[0]) )

All 3-grams in sales associate:
['sal', 'ale', 'les', 'es ', 's a', ' as', 'ass', 'sso', 'soc', 'oci', 'cia', 'iat', 'ate']


In [59]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix = vectorizer.fit_transform(clean_job_titles_filtered)

print(len(vectorizer.get_feature_names()))

9125


In [60]:
print(clean_job_titles_filtered[0])
print( tf_idf_matrix[0] )

sales associate
  (0, 1202)	0.2629246497962331
  (0, 3570)	0.28059702929542235
  (0, 1708)	0.21463684724075915
  (0, 5433)	0.28027840461919223
  (0, 6842)	0.28019312050070816
  (0, 6915)	0.2760702445118371
  (0, 1192)	0.23772697026601597
  (0, 60)	0.25632579122519306
  (0, 6606)	0.32820228587555406
  (0, 2629)	0.2861859371771339
  (0, 4374)	0.29647407670993225
  (0, 1044)	0.2933294289944778
  (0, 6652)	0.2949793618269292


In [61]:
import numpy as np
from scipy.sparse import csr_matrix
import sparse_dot_topn.sparse_dot_topn as ct

def awesome_cossim_top(A, B, ntop, lower_bound=0):
    # force A and B as a CSR matrix.
    # If they have already been CSR, there is no overhead
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
 
    idx_dtype = np.int32
 
    nnz_max = M*ntop
 
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)

    ct.sparse_dot_topn(
        M, N, np.asarray(A.indptr, dtype=idx_dtype),
        np.asarray(A.indices, dtype=idx_dtype),
        A.data,
        np.asarray(B.indptr, dtype=idx_dtype),
        np.asarray(B.indices, dtype=idx_dtype),
        B.data,
        ntop,
        lower_bound,
        indptr, indices, data)

    return csr_matrix((data,indices,indptr),shape=(M,N))

In [62]:
# import time
# t1 = time.time()
# matches = awesome_cossim_top(tf_idf_matrix, tf_idf_matrix.transpose(), 10, 0.7)
# t = time.time()-t1
# print("SELFTIMED:", t)

In [63]:
from sparse_dot_topn import awesome_cossim_topn
import time

t1 = time.time()

matches = awesome_cossim_topn(tf_idf_matrix, tf_idf_matrix.transpose(), 10, 0.7, use_threads=True, n_jobs=11)

t = time.time()-t1
print("SELFTIMED:", t)

SELFTIMED: 36.84464693069458


In [64]:
def get_matches_df(sparse_matrix, name_vector, top=100):
    non_zeros = sparse_matrix.nonzero()
    
    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]
    
    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size
    
    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similairity = np.zeros(nr_matches)
    
    for index in range(0, nr_matches):
        left_side[index] = name_vector[sparserows[index]]
        right_side[index] = name_vector[sparsecols[index]]
        similairity[index] = sparse_matrix.data[index]
    
    return pd.DataFrame({'left_side': left_side,
                          'right_side': right_side,
                           'similairity': similairity})

In [65]:
matches_df = get_matches_df(matches, clean_job_titles_filtered, top=False)
# matches_df = matches_df[matches_df['similairity'] < 0.99999] # Remove all exact matches
matches_df.sample(20)

,left_side,right_side,similairity
88157,assistant management,assistant manager management assistant,0.907061
496763,associate retail sales,retail sales,0.807787
573794,cicu,cicu,1.000000
504789,property claims associate adjuster,claims adjuster,0.746520
388886,personnel security administrator,security administration,0.710243
519131,especialista almacen,especialista almacén,0.765514
478817,walmart night stocker,walmart over night stocker,0.814992
266827,software engineer project,software engineer project manager,0.920273
410079,loss prevention specialist trainer,loss prevention specialist store,0.804219
300149,graphic designer web developer,graphic designer designer,0.707429


In [66]:
matches_df.sort_values(['similairity'], ascending=True).head(10)

,left_side,right_side,similairity
603779,lowe employee,associate employee,0.700000
398089,garden associate receiving,sales associate receiving associate,0.700001
629285,clerk assembler,assemble,0.700001
439248,técnico eletrônica telecomunicações,técnico telecomunicações pleno,0.700001
281329,técnico telecomunicações pleno,técnico eletrônica telecomunicações,0.700001
337733,social media team,social media manager,0.700002
48399,social media manager,social media team,0.700002
502810,salesman merchandising specialist,merchandising sales,0.700002
608257,ammunition specialist,communication specialist,0.700002
446754,lab support engineer,associate support engineer,0.700002


In [67]:
filtered_jobs = matches_df.loc[matches_df.left_side.isin(common_tech_titles),["left_side","right_side"]]
print(filtered_jobs.shape)

pd.set_option("display.max_colwidth",500)
display(filtered_jobs.groupby("left_side")["right_side"].apply(list).reset_index())
pd.reset_option("display.max_colwidth")


(1198, 2)


,left_side,right_side
0,application developer,"[application developer, application developer, application developer sap, application developer team, application developer associate, application developer analyst, application developer specialist, net application developer, rational application developer, application developer intern, application developer, application developer, application developer sap, application developer team, application developer associate, application developer analyst, application developer specialist, net appl..."
1,application support analyst,"[application support analyst, application support analyst, applications support analyst, technical application support analyst, clinical application support analyst, application support, business application support analyst, application production support analyst, production application support analyst, application technical support analyst, application support analyst, application support analyst, applications support analyst, technical application support analyst, clinical application supp..."
2,applications engineer,"[applications engineer, applications engineer, applications engineer sales engineering, applications engineer intern, application operations engineer, application sales engineer, application engineer, technical applications engineer, applications engineering intern, applications systems engineer, applications engineer, applications engineer, applications engineer sales engineering, applications engineer intern, application operations engineer, application sales engineer, application engineer..."
3,associate developer,"[associate developer, associate developer, developer associate, associate development, developer developer, developer, software associate developer, associate development manager, store developer, associate software developer, associate developer, associate developer, developer associate, associate development, developer developer, developer, software associate developer, associate development manager, store developer, associate software developer]"
4,chief information officer,"[chief information officer, chief information officer, chief information office, chief information officer cio, chief information security officer, information officer, chief information systems, chief information officer, chief information officer, chief information office, chief information officer cio, chief information security officer, information officer, chief information systems]"
...,...,...
65,technology systems administrator,"[technology systems administrator, technology systems administrator, technology system administrator, systems administrator technology consultant, technology specialist system administrator, systems administrator, systems administrator, technology administrator, tier systems administrator, system administrator technology, technology systems administrator, technology systems administrator, technology system administrator, systems administrator technology consultant, technology specialist syst..."
66,telecommunications specialist,"[telecommunications specialist, telecommunications specialist, telecommunication specialist, telecommunications operations specialist, communications specialist, telecommunications, telecommunications sales, technology telecommunications specialist, telecommunications specialist project manager, telecommunications manager, telecommunications specialist, telecommunications specialist, telecommunication specialist, telecommunications operations specialist, communications specialist, telecommun..."
67,web administrator,"[web administrator, web administrator, lab administrator, web content administrator, web administrator, web administrator, lab administrator, web content administrator]"
68,web developer,"[web developer, web developer, web developer developer, web developer analyst, web developer intern, web application developer web developer, net web developer, associate web developer, assistant web d

In [68]:
sum(reviews.clean_job_title.isin(filtered_jobs.right_side.tolist()))

42686

## TF-IDF with K-Means

In [69]:

clean_job_titles = reviews.clean_job_title.unique().tolist()
print(len(clean_job_titles))

506654


In [102]:
import re

def ngrams(string, n=3):
    #string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

print('All 3-grams in {}:'.format(common_tech_titles[0]))
print( ngrams(common_tech_titles[0]) )

All 3-grams in cloud system engineer:
['clo', 'lou', 'oud', 'ud ', 'd s', ' sy', 'sys', 'yst', 'ste', 'tem', 'em ', 'm e', ' en', 'eng', 'ngi', 'gin', 'ine', 'nee', 'eer']


In [72]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
tf_idf_matrix = vectorizer.fit_transform(common_tech_titles)

print(len(vectorizer.get_feature_names()))

340


In [79]:
from sklearn.neighbors import NearestNeighbors
import time

nbrs = NearestNeighbors(n_neighbors=1, n_jobs=-1).fit(tf_idf_matrix)

In [80]:
def getNearestN(query):
    queryTFIDF_ = vectorizer.transform(query)
    distances, indices = nbrs.kneighbors(queryTFIDF_)
    return distances, indices

t1 = time.time()
print('getting nearest n...')
distances, indices = getNearestN(clean_job_titles)
t = time.time()-t1
print("COMPLETED IN:", t)

getting nearest n...
COMPLETED IN: 7.010178804397583


In [83]:
print('finding matches...')
kmeans_matches = []

for i,j in enumerate(indices):
    temp = [round(distances[i][0],2), common_tech_titles.Job_Title.tolist()[j[0]],clean_job_titles[i]]
    kmeans_matches.append(temp)
print('Building data frame...')  
kmeans_matches = pd.DataFrame(kmeans_matches, columns=['Match confidence (lower is better)','left_side','right_side'])
print('Done')

finding matches...
Building data frame...
Done


In [84]:
idx = kmeans_matches["Match confidence (lower is better)"] < .4
kmeans_filtered_jobs = kmeans_matches.loc[idx,["left_side","right_side"]]
print(sum(idx))

pd.set_option("display.max_colwidth",700)
display( kmeans_filtered_jobs.groupby("left_side").right_side.apply(list).reset_index() )
pd.reset_option("display.max_colwidth")

1501


,left_side,right_side
0,application developer,"[network administrator, network domino administrator, network administator, ibm network administrator, network adminstrator, mis windows network administrator, network administrator voip, network administrator gboss, network administrator lan wan, linux network administrator, lan network administrator, network administrator lll, network administrador, payback network administrator]"
1,application support analyst,"[system architect, pega system architect, sub system architect, edo system architect, bpm system architect]"
2,applications engineer,"[programmer analyst, programmer analysis, cobol programmer analyst, programmer analyst bmw china, ahcis programmer analyst, sas programmer analyst, programmer analyst full time, programmer analyst builder, its programmer analyst, computer systems analyst, programmer analyst sap abap, programmer annalyst, team programmer analyst, programmer analyst powerbuilder, programmer analyst sis, programmer analyst testing, programmer analyst level, programmer analyst programmer, india programmer analyst, programmer anaylyst, programmer analyst oracle retail, programmer analyst test, programmer analyst trainee, programmer analyst trainee pat, woking programmer analyst, programmer analyst role, progr..."
3,associate developer,"[webmaster, webmaster sav, tester webmaster]"
4,chief technology officer,"[help desk technician, itd help desk technician, intern help desk technician, global help desk technician, help desk field technician, tier help desk technician, help desk technician richter library, csr help desk technician, level help desk technician, spot help desk technician, help desk technician level, level one help desk technician, seior help desk technician, windows installer help desk technician, help desk technician internship, capital one help desk technician, microsoft global help desk technician, pss help desk technician, help desk technician tier, nmci help desk technician, help desk technician alaska airlines, help desk technician siroms, advanced help desk technician, hel..."
5,cloud architect,[cloud system engineer]
6,cloud consultant,"[technology analyst, people technology analyst, technology planning analyst, technology pmo analyst, technology audit analyst, technology analyst gsc audit, technology advisory analyst, technology analyst salary perm, technology ananlyst, technology analysis, chennai technology analyst, technology analayst, sap technology analyst, technology analyst oracle dba, technology analyst full time, role technology analyst, technology analyst dba, technology analyst sap hcm, technology analyst team, test technology analyst, technology analyst pune, technology analyst tcs, technology analyst technology, technology analyst from11, technology analyts, technology analyst unix, india technology analys..."
7,cloud product project manager,"[technology coordinator, oncology coordinator, assistive technology coordinator, technology team safety coordinator, technology inventory coordinator, ibm field technology coordinator, business technology coordinator, valo technology coordinator, technology audit coordinator, technology coordinator tier, pathology coordinator, technology coordinator bsc5, technology coach coordinator, benadmin technology coordinator, local technology coordinator, group technology coordinator, mcsa technology coordinator, regional technology coordinator, surgery technology coordinator, intern technology coordinator, radiology coordinator]"
8,cloud services developer,"[network systems administrator, bhqn network systems administrator]"
9,cloud software network engineer,"[telecommunications specialist, telecommunications repair specialist, telecommunications account specialist, military telecommunications specialist, analyst telecommunications specialist, telecommunications systems specialist, telecommunications specialist csr]"


In [85]:
sum(reviews.clean_job_title.isin(kmeans_filtered_jobs.right_side.tolist()))

36342

Merge both datsets

In [87]:

finalized_job_list = pd.concat([filtered_jobs, kmeans_filtered_jobs]).drop_duplicates()

print(finalized_job_list.shape)

display( finalized_job_list.head() )

# finalized_job_list.to_csv("../data/finalized_job_list.csv", index=False)


(2052, 2)
(2052, 3)


,job_title,common_job_title,Job_Family
0,software engineer,software engineer,Software/Application Developer
3,software engineer software engineer,software engineer,Software/Application Developer
6,software engineering,software engineer,Software/Application Developer
9,software engineer engineer,software engineer,Software/Application Developer
12,engineer software engineer,software engineer,Software/Application Developer


In [89]:
sum(reviews.clean_job_title.isin(finalized_job_list.job_title.tolist()))

45857

Remove Jobs...

In [90]:
filter_titles = finalized_job_list.job_title.tolist()

exclude_keywords = [
    "camp"
    ,"tutor"
    ,"professor"
    ,"entry"
    ,"entries"
    ,"teacher"
    ,"audio"
    ,"civil"
    ,"facility"
    ,"facilities"
    ,"industrial"
    ,"metals"  
    ,"lab"
]

def excludeJobTItles(job):
    tokens = str(job).split(" ")
    return len( [ word for word in tokens if word in exclude_keywords ] ) == 0

filter_titles_2 = list( filter(excludeJobTItles, filter_titles) )

In [91]:
sum(reviews.clean_job_title.isin(filter_titles_2))

45843

In [ ]:
import pandas as pd

# pd.DataFrame(filter_titles_2,columns=["clean_job_title"]).to_csv("../data/filter_job_titles.csv",index=False)

Add Job Family

In [113]:
finalized_job_list = pd.read_csv("../data/finalized_job_list.csv")

common_tech_titles = pd.read_csv("../data/job_titles.txt")

common_tech_titles["Job_Title"] = common_tech_titles.Job_Title.apply(cleanTitle).tolist()

print( finalized_job_list.shape )

finalized_job_list = finalized_job_list.merge(common_tech_titles, how="left", left_on="left_side",right_on="Job_Title")\
                        .rename(columns={"right_side":"job_title","Job_Title":"common_job_title"})

del finalized_job_list["left_side"]

finalized_job_list.drop_duplicates(inplace=True)

print( finalized_job_list.shape )

finalized_job_list.to_csv("../data/finalized_job_list job_family.csv")


(1855, 2)
(1855, 3)
